In [1]:
import pandas as pd
import numpy as np

In [2]:
#Читаем данные
oldData = pd.read_excel("/content/drive/MyDrive/Работа РСХБ/Выгрузки/выгрузка_21.10 постарше.xlsx")
newData = pd.read_excel("/content/drive/MyDrive/Работа РСХБ/Выгрузки/выгрузка_28.10.xlsx")
excelOrder = pd.read_excel(
    "/content/drive/MyDrive/Работа РСХБ/Копия Наполнение каталога.xlsx",
    sheet_name = None                
)
#Дата, с которой сравниваем
prevDate = pd.to_datetime("10.21.2021")#Месяц, число, год
#Дата формирования отчёта
nowDate = pd.to_datetime("10.28.2021")

In [3]:
#Нормируем данные из выгрузок
oldData = oldData.astype({'ИНН организации': np.int})
newData = newData.astype({'ИНН организации': np.int})
oldData["Дата регистрации организации на платформе \"Свое фермерство \""] \
= pd.to_datetime(oldData["Дата регистрации организации на платформе \"Свое фермерство \""])
newData["Дата регистрации организации на платформе \"Свое фермерство \""] \
= pd.to_datetime(newData["Дата регистрации организации на платформе \"Свое фермерство \""])
#В сводном файле удалим строку со статистикой, заполним пустые ячейки нулями, приведём ИНН к инту
excelOrder['ЖИВОТНЫЕ'] = (excelOrder['ЖИВОТНЫЕ'].drop(index=[0]).fillna(value=0)).astype({'ИНН организации': np.int}).reset_index(drop=True)
excelOrder['ПОСАД МАТЕР'] = (excelOrder['ПОСАД МАТЕР'].drop(index=[0]).fillna(value=0)).astype({'ИНН организации': np.int}).reset_index(drop=True)
excelOrder['СЕМЕНА'] = (excelOrder['СЕМЕНА'].drop(index=[0]).fillna(value=0)).astype({'ИНН организации': np.int}).reset_index(drop=True)

In [25]:
# category = 'Сельскохозяйственные животные'
def create_order(category):
  print(f"КАТЕГОРИЯ {category.upper()}")
  oldDataINN = oldData[(oldData['Категория товара уровня 1'] == f"{category}")]["ИНН организации"].unique()
  newDataINN = newData[(newData['Категория товара уровня 1'] == f"{category}")]["ИНН организации"].unique() 
  generalOldINN = excelOrder['ЖИВОТНЫЕ']['ИНН организации'].unique()#Продавцы, имеющиеся в базе на сегодня

  prevDif = np.setdiff1d(newDataINN, oldDataINN)#Появились в сравнении с прошлой выгрузкой
  print(f"Было {len(oldDataINN)} производителей и поставщиков")
  print(f"Стало {len(newDataINN)} производителей и поставщиков")
  print(f"Прирост составил {len(newDataINN) - len(oldDataINN)} производителей и поставщиков")

  generalDif = np.setdiff1d(prevDif, generalOldINN)#Продавцы, которых ещё не было в базе
  newSellersGeneral = newData[(newData["ИНН организации"].isin(generalDif)) \
                        & (newData['Категория товара уровня 1'] == f"{category}")]
  #Продавцы, которых нет в базе и дата регистрации - 2021 год
  newSellers = newSellersGeneral[newSellersGeneral["Дата регистрации организации на платформе \"Свое фермерство \""] >= pd.to_datetime("01.01.2021")]

  print(f"За данный период в категории '{category}' появилось {newSellers.shape[0]} новых продавца или производителя.")

  #Активировавштеся продавцы
  activatedSellersINN = np.setdiff1d(prevDif, newSellers['ИНН организации'])#Производители, которые есть в базе, или те, которых ещё нет, но год регистрации раньше текущего
  print(f"Активировалось {len(activatedSellersINN)} продавцов или производителей.")
            
  dif = np.setdiff1d(oldDataINN, newDataINN)#пропали в новой выгрузке
  print(f"Также за период из категории '{category}' ушло {len(dif)} продавцов и производителей.")
  #Прирост количества товаров
  oldSum = oldData[(oldData['Категория товара уровня 1'] == f"{category}")]["Количество товаров у поставщика"].sum()
  newSum = newData[(newData['Категория товара уровня 1'] == f"{category}")]["Количество товаров у поставщика"].sum()
  print(f"В категории было {oldSum} товаров, стало {newSum} товаров. Прирост составил {newSum - oldSum}.")

  #Сколько добавили новые участники
  newcomersContrib = newSellers["Количество товаров у поставщика"].sum()
  print(f"Из {newSum - oldSum} новых товаров {newcomersContrib} добавили новые участники площадки.")

  #Список новых участников
  print()
  print("НОВЫЕ УЧАСТНИКИ")
  for inn in (newSellers["ИНН организации"]):
    name = newSellers.loc[newSellers["ИНН организации"] == inn]["Наименование организации"].values
    print(f"{inn} {name[0]}")
  #Список активизировавшихся участников
  if(len(activatedSellersINN) > 0):
    print()
    print("АКТИВИЗИРОВАВШИЕСЯ УЧАСТНИКИ")
    for inn in (newData[(newData["ИНН организации"].isin(activatedSellersINN)) &\
              (newData['Категория товара уровня 1'] == f"{category}")]):
      name = newSellers.loc[newSellers["ИНН организации"] == inn]["Наименование организации"].values
      print(f"{inn} {name[0]}")
    print()
  else:
    print()
    print("АКТИВИЗИРОВАВШИХСЯ УЧАСТНИКОВ НЕТ\n")

In [26]:
create_order("Сельскохозяйственные животные")
create_order("Семена")
create_order("Посадочные материалы")

КАТЕГОРИЯ СЕЛЬСКОХОЗЯЙСТВЕННЫЕ ЖИВОТНЫЕ
Было 617 производителей и поставщиков
Стало 623 производителей и поставщиков
Прирост составил 6 производителей и поставщиков
За данный период в категории 'Сельскохозяйственные животные' появилось 6 новых продавца или производителя.
Активировалось 0 продавцов или производителей.
Также за период из категории 'Сельскохозяйственные животные' ушло 0 продавцов и производителей.
В категории было 1391 товаров, стало 1414 товаров. Прирост составил 23.
Из 23 новых товаров 24 добавили новые участники площадки.

НОВЫЕ УЧАСТНИКИ
613900202897 Глава крестьянского (фермерского) хозяйства Зубенко Андрей Николаевич
613901928730 Глава крестьянского (фермерского) хозяйства Ефремов Роман Алексеевич
240900016237 Глава крестьянского (фермерского) хозяйства Генза Олег Саламонович
81100524868 Глава крестьянского (фермерского) хозяйства Шукургалиев Кобланды Захарович
3917508263 ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ФЕРМА"
3849012202 ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕНН

In [ ]:
#Обновим листы для животных, семян, посадочных материалов

#Животные
newAnimals = newData[(newData['Категория товара уровня 1'] == "Животные") \
                       | (newData['Категория товара уровня 1'] == "Сельскохозяйственные животные")].drop(
            columns=[
                    #  "Дата регистрации организации на платформе \"Свое фермерство \"",
                     "Идентификатор организации",
                     "Признак b2b",
                     "Признак b2c",
                     "Категория товара уровня 1"
            ])
refreshAnimals = pd.merge(
    excelOrder['ЖИВОТНЫЕ'],
    newAnimals,
    how="outer",
    on=[
        "ИНН организации", 
        "Наименование региона регистрации",
    ]
).fillna(value=0).drop_duplicates(keep='first').rename(columns={'Количество товаров у поставщика': nowDate})
refreshAnimals['Наименование организации'] = [x if x != "" else refreshAnimals['Наименование организации_x']\
                                                for x in refreshAnimals['Наименование организации_y']] 
refreshAnimals['Наименование организации_x'] = refreshAnimals["Наименование организации"]
refreshAnimals = refreshAnimals.drop(columns=["Наименование организации_y", "Наименование организации"])
refreshAnimals = refreshAnimals.rename(columns={'Наименование организации_x':'Наименование организации'})
#Вставим строку со статистической информацией
titles = refreshAnimals.columns.values[3:len(refreshAnimals.columns.values)]
for i in range(len(titles)):
  if titles[i] == "Дата регистрации организации на платформе \"Свое фермерство \"":
    titles = np.delete(titles, i)
    break
tmpDict = {}
for title in titles:
  tmpDict[title] = [refreshAnimals[title].sum()]
tmpDict["Наименование организации"] = len(refreshAnimals["ИНН организации"].unique())
statisRow = pd.DataFrame(data=tmpDict, columns=refreshAnimals.columns.values)
statisRow.fillna(value="")
refreshAnimals = pd.concat([statisRow, refreshAnimals]).fillna(value="").reset_index(drop=True)
#Поставим колонку с датой регистрации в нормальное место
# refreshAnimals.insert(1, "Дата регистрации", refreshAnimals["Дата регистрации организации на платформе \"Свое фермерство \""])
# refreshAnimals = refreshAnimals.drop(columns=["Дата регистрации организации на платформе \"Свое фермерство \""])


#Семена
# newSeeds = newData[(newData['Категория товара уровня 1'] == "Семена")].drop(
#             columns=[
#                     #  "Дата регистрации организации на платформе \"Свое фермерство \"",
#                      "Идентификатор организации",
#                      "Признак b2b",
#                      "Признак b2c",
#                      "Категория товара уровня 1"
#             ])
# refreshSeeds = pd.merge(
#     excelOrder['СЕМЕНА'],
#     newSeeds,
#     how="outer",
#     on=[
#         "ИНН организации", 
#         "Наименование региона регистрации",
#     ]
# ).fillna(value=0).drop_duplicates(keep='first').rename(columns={'Количество товаров у поставщика': nowDate})
# refreshSeeds['Наименование организации'] = [x if x != "" else refreshSeeds['Наименование организации_x']\
#                                                 for x in refreshSeeds['Наименование организации_y']] 
# refreshSeeds['Наименование организации_x'] = refreshSeeds["Наименование организации"]
# refreshSeeds = refreshSeeds.drop(columns=["Наименование организации_y", "Наименование организации"])
# refreshSeeds = refreshSeeds.rename(columns={'Наименование организации_x':'Наименование организации'})
# #Вставим строку со статистической информацией
# titles = refreshSeeds.columns.values[3:len(refreshSeeds.columns.values)]
# for i in range(len(titles)):
#   if titles[i] == "Дата регистрации организации на платформе \"Свое фермерство \"":
#     titles = np.delete(titles, i)
#     break
# tmpDict = {}
# for title in titles:
#   tmpDict[title] = [refreshSeeds[title].sum()]
# tmpDict["Наименование организации"] = len(refreshSeeds["ИНН организации"].unique())
# statisRow = pd.DataFrame(data=tmpDict, columns=refreshSeeds.columns.values)
# statisRow.fillna(value="")
# refreshSeeds = pd.concat([statisRow, refreshSeeds]).fillna(value="").reset_index(drop=True)
# #Поставим колонку с датой регистрации в нормальное место
# # refreshSeeds.insert(1, "Дата регистрации", refreshSeeds["Дата регистрации организации на платформе \"Свое фермерство \""])
# # refreshSeeds = refreshSeeds.drop(columns=["Дата регистрации организации на платформе \"Свое фермерство \""])


# #Посадочные материалы
# newSeedlings = newData[(newData['Категория товара уровня 1'] == "Посадочные материалы")].drop(
#             columns=[
#                     #  "Дата регистрации организации на платформе \"Свое фермерство \"",
#                      "Идентификатор организации",
#                      "Признак b2b",
#                      "Признак b2c",
#                      "Категория товара уровня 1"
#             ])
# refreshSeedlings = pd.merge(
#     excelOrder['ПОСАД МАТЕР'],
#     newSeedlings,
#     how="outer",
#     on=[
#         "ИНН организации", 
#         "Наименование региона регистрации",
#     ]
# ).fillna(value=0).drop_duplicates(keep='first').rename(columns={'Количество товаров у поставщика': nowDate})
# refreshSeedlings['Наименование организации'] = [x if x != "" else refreshSeedlings['Наименование организации_x']\
#                                                 for x in refreshSeedlings['Наименование организации_y']] 
# refreshSeedlings['Наименование организации_x'] = refreshSeedlings["Наименование организации"]
# refreshSeedlings = refreshSeedlings.drop(columns=["Наименование организации_y", "Наименование организации"])
# refreshSeedlings = refreshSeedlings.rename(columns={'Наименование организации_x':'Наименование организации'})
# #Вставим строку со статистической информацией
# titles = refreshSeedlings.columns.values[3:len(refreshSeedlings.columns.values)]
# for i in range(len(titles)):
#   if titles[i] == "Дата регистрации организации на платформе \"Свое фермерство \"":
#     titles = np.delete(titles, i)
#     break
# tmpDict = {}
# for title in titles:
#   tmpDict[title] = [refreshSeedlings[title].sum()]
# tmpDict["Наименование организации"] = len(refreshSeedlings["ИНН организации"].unique())
# statisRow = pd.DataFrame(data=tmpDict, columns=refreshSeedlings.columns.values)
# statisRow.fillna(value="")
# refreshSeedlings = pd.concat([statisRow, refreshSeedlings]).fillna(value="").reset_index(drop=True)
#Поставим колонку с датой регистрации в нормальное место
# refreshSeedlings.insert(1, "Дата регистрации", refreshSeedlings["Дата регистрации организации на платформе \"Свое фермерство \""])
# refreshSeedlings = refreshSeedlings.drop(columns=["Дата регистрации организации на платформе \"Свое фермерство \""])


In [ ]:
# # refreshSeeds['ИНН организации'].unique()
# refreshSeeds
with pd.ExcelWriter(f"/content/drive/MyDrive/Работа РСХБ/Общая папка/Наполнение каталога {nowDate.strftime('%d.%m-%Y')}.xlsx") as writer:  

    refreshSeeds.to_excel(writer, sheet_name='СЕМЕНА',index=False)

    refreshSeedlings.to_excel(writer, sheet_name='ПОСАД МАТЕР',index=False)
    
    refreshAnimals.to_excel(writer, sheet_name='ЖИВОТНЫЕ',index=False)

In [ ]:
newSellers